# Practicing imputation with cars dataset 

In [1]:
#Importing the classics
import pandas as pd 
import numpy as np 

#Importing sklearn tools
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

from sklearn.impute import SimpleImputer, MissingIndicator

In [2]:
df = pd.read_csv('Mock_CAR_GENDER_MAKE_PRICE.csv', header = 0, index_col = 0)
df.shape

(1000, 3)

In [3]:
df.head(15)

,gender,carmake,price
id,,,
1,Male,Honda,16907.46
2,Female,Chevrolet,25298.09
3,Female,Mazda,21812.35
4,Male,Honda,22583.60
5,Female,Aston Martin,28987.44
6,Male,Mercury,24213.82
7,Male,Chevrolet,29778.91
8,Female,Chevrolet,28993.37
9,Female,Nissan,21563.51


Lets replace the ID with zero index

In [4]:
df = df.reset_index(drop = True)

In [5]:
df.head()

,gender,carmake,price
0,Male,Honda,16907.46
1,Female,Chevrolet,25298.09
2,Female,Mazda,21812.35
3,Male,Honda,22583.60
4,Female,Aston Martin,28987.44


Replacing the carmake labels to omit spaces and all lower case e.g. 'Aston Martin' -> 'astonmartin'

In [6]:
df.carmake = df.carmake.replace('\s+', '',regex=True)
df.carmake = df.carmake.str.lower()

Replacing gender column to be female true column

In [7]:
#Relabel sex
df[['gender']] = df[['gender']].replace({'Female':1.,  'Male':0.})
df.head(15)

,gender,carmake,price
0,0.0,honda,16907.46
1,1.0,chevrolet,25298.09
2,1.0,mazda,21812.35
3,0.0,honda,22583.60
4,1.0,astonmartin,28987.44
5,0.0,mercury,24213.82
6,0.0,chevrolet,29778.91
7,1.0,chevrolet,28993.37
8,1.0,nissan,21563.51
9,0.0,mercury,23162.97


In [8]:
df.describe()

,gender,price
count,1000.000000,801.000000
mean,0.481000,20027.687478
std,0.499889,5689.499086
min,0.000000,10031.760000
25%,0.000000,15360.720000
50%,0.000000,20161.840000
75%,1.000000,24977.690000
max,1.000000,29917.250000


## Removing all nan values 

In [9]:
df1 = df.dropna(inplace=False)
df_inter = df.fillna(0)
df_nan = df_inter[df_inter['price'] == 0]

df_nan = df_nan.drop('price', axis = 1)
df_nan = df_nan.reset_index(drop = True)
df_nan.describe()

,gender
count,199.000000
mean,0.517588
std,0.500951
min,0.000000
25%,0.000000
50%,1.000000
75%,1.000000
max,1.000000


In [10]:
sum(df1['carmake'].str.contains('smart'))

0

Notice that the training data will not contain any smart cars... 

In [11]:
df1.head()

,gender,carmake,price
0,0.0,honda,16907.46
1,1.0,chevrolet,25298.09
2,1.0,mazda,21812.35
3,0.0,honda,22583.60
4,1.0,astonmartin,28987.44


In [12]:
df1.tail()

,gender,carmake,price
993,0.0,audi,29742.98
994,1.0,oldsmobile,19087.81
995,0.0,chevrolet,14743.35
996,1.0,ford,13630.00
998,0.0,lotus,15388.68


In [13]:
df1.describe(include = 'all')

,gender,carmake,price
count,801.000000,801,801.000000
unique,NaN,54,NaN
top,NaN,ford,NaN
freq,NaN,78,NaN
mean,0.471910,NaN,20027.687478
std,0.499522,NaN,5689.499086
min,0.000000,NaN,10031.760000
25%,0.000000,NaN,15360.720000
50%,0.000000,NaN,20161.840000
75%,1.000000,NaN,24977.690000


## Very simple imputation method: Replace missing values with mean of entire comumn 

We ignore the gategorical data here, just replacing every missing value with the overall mean, we dont distingush between car type or gender. 

In [14]:
# Create an imputer that fills missing values with mean column values - could just as well have chosen, median, mode, or just some hand selected value such as 0. using replace(np.nan, 0)
imputer = SimpleImputer(strategy='mean')

In [15]:
# split dataset into inputs and outputs
df_values = df.values
X = df_values[:,0:2]
y = df_values[:,2]

In [16]:
y.shape

(1000,)

In [17]:
y_reshape = y.reshape(-1, 1)
y_reshape.shape

(1000, 1)

In [18]:
transformed_y = imputer.fit_transform(y_reshape)

In [19]:
df2part1 = pd.DataFrame(X, columns = ['female', 'make'])
df2part2 = pd.DataFrame(transformed_y, columns = ['price'])
df2 = df2part1.join(df2part2)
df2.head(15)

,female,make,price
0,0,honda,16907.460000
1,1,chevrolet,25298.090000
2,1,mazda,21812.350000
3,0,honda,22583.600000
4,1,astonmartin,28987.440000
5,0,mercury,24213.820000
6,0,chevrolet,29778.910000
7,1,chevrolet,28993.370000
8,1,nissan,21563.510000
9,0,mercury,23162.970000


In [20]:
df2.describe(include = 'all')

,female,make,price
count,1000.0,1000,1000.000000
unique,2.0,55,NaN
top,0.0,ford,NaN
freq,519.0,100,NaN
mean,NaN,NaN,20027.687478
std,NaN,NaN,5091.389017
min,NaN,NaN,10031.760000
25%,NaN,NaN,16359.345000
50%,NaN,NaN,20027.687478
75%,NaN,NaN,23689.645000


## Less simple imputation method: replacing price values based on the categorical data

Idea: lets sepearate the dataframe. One dataframe with no missing values and the other will contain all the missing values. 

We have already created the first one: df1. lets rename it: df_train. 
This would not be the best method if multiple columns had missing values, nor if we did not have sufficient number of rows without missing columns. 

Adding dummy variables to our categorical data i.e. carmake. We use TF-IDF.

In [21]:
#carmake 
tfidf = TfidfVectorizer()
make_values = tfidf.fit_transform(df.carmake).toarray()

In [22]:
col_names = tfidf.get_feature_names()
make_values_total_df = pd.DataFrame(data = make_values, columns = col_names) 

In [23]:
make_values_total_df.head()

,acura,astonmartin,audi,bentley,benz,bmw,buick,cadillac,chevrolet,chrysler,...,saab,saturn,scion,smart,subaru,suzuki,tesla,toyota,volkswagen,volvo
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [24]:
df_tfidf = df.join(make_values_total_df)
df_tfidf = df_tfidf.drop('carmake', axis = 1)
df_tfidf = df_tfidf[[c for c in df_tfidf if c not in ['price']] + ['price']]
df_tfidf.head()

,gender,acura,astonmartin,audi,bentley,benz,bmw,buick,cadillac,chevrolet,...,saturn,scion,smart,subaru,suzuki,tesla,toyota,volkswagen,volvo,price
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,16907.46
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25298.09
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,21812.35
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,22583.60
4,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,28987.44


Creating the training dataset 

In [25]:
df_train = df_tfidf.dropna(inplace=False)
df_train = df_train.reset_index(drop = True)
df_train.describe()

,gender,acura,astonmartin,audi,bentley,benz,bmw,buick,cadillac,chevrolet,...,saturn,scion,smart,subaru,suzuki,tesla,toyota,volkswagen,volvo,price
count,801.000000,801.000000,801.000000,801.000000,801.000000,801.000000,801.000000,801.000000,801.000000,801.000000,...,801.000000,801.000000,801.0,801.000000,801.000000,801.000000,801.000000,801.000000,801.000000,801.000000
mean,0.471910,0.011236,0.002497,0.019975,0.008739,0.026483,0.022472,0.024969,0.024969,0.069913,...,0.007491,0.002497,0.0,0.013733,0.011236,0.001248,0.041199,0.037453,0.012484,20027.687478
std,0.499522,0.105468,0.049938,0.140002,0.093132,0.134342,0.148305,0.156127,0.156127,0.255159,...,0.086278,0.049938,0.0,0.116452,0.105468,0.035333,0.198873,0.189988,0.111103,5689.499086
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,10031.760000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,15360.720000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,20161.840000
75%,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,24977.690000
max,1.000000,1.000000,1.000000,1.000000,1.000000,0.707107,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,0.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,29917.250000


In [26]:
df_train.tail()

,gender,acura,astonmartin,audi,bentley,benz,bmw,buick,cadillac,chevrolet,...,saturn,scion,smart,subaru,suzuki,tesla,toyota,volkswagen,volvo,price
796,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,29742.98
797,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,19087.81
798,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,14743.35
799,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,13630.00
800,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,15388.68


In [27]:
df_tfidf[['price']] = df_tfidf[['price']].replace(np.NaN, 0)
df_test = df_tfidf[df_tfidf.price == 0.]
df_test = df_test.drop('price', axis = 1)
df_test = df_test.reset_index(drop = True)
df_test.head()

,gender,acura,astonmartin,audi,bentley,benz,bmw,buick,cadillac,chevrolet,...,saab,saturn,scion,smart,subaru,suzuki,tesla,toyota,volkswagen,volvo
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Now that we have created the training and test set we can create our pipeline 

In [28]:
X_train = df_train.iloc[:,df_train.columns!='price'].values

In [29]:
y_train = df_train['price'].values

In [30]:
X_test = df_test.values


In [31]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import GridSearchCV

In [32]:
linreg = LinearRegression()
linreg.fit(X_train,y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

In [33]:
linreg.get_params()

{'copy_X': True, 'fit_intercept': True, 'n_jobs': None, 'normalize': False}

In [34]:
linreg.score(X_train, y_train)

0.076893384964142

In [35]:
#grid_search= GridSearchCV(estimator=linreg, cv=5, n_jobs=-1)
#grid_search=grid_search.fit(X,y)

In [36]:
#best_accuracy=grid_search.best_score_
#best_parameters=grid_search.best_params_
#print (best_accuracy)
#print (best_parameters)

In [37]:
linreg.predict(X_test)

array([2.08290000e+04, 2.00720000e+04, 2.01980000e+04, 1.96000000e+04,
       1.83720000e+04, 2.05180000e+04, 2.08680000e+04, 1.88510000e+04,
       2.09920000e+04, 2.00720000e+04, 2.00260000e+04, 2.08280000e+04,
       1.73250000e+04, 2.21400000e+04, 2.21400000e+04, 2.16680000e+04,
       2.00260000e+04, 1.86040000e+04, 2.08280000e+04, 1.95540000e+04,
       2.23620000e+04, 1.81450000e+04, 1.86170000e+04, 2.00720000e+04,
       2.12120000e+04, 1.90080000e+04, 1.49410000e+04, 1.77970000e+04,
       2.06620000e+04, 2.08680000e+04, 2.21400000e+04, 1.83720000e+04,
       1.81320000e+04, 2.01980000e+04, 5.30991112e+15, 2.12120000e+04,
       2.09900000e+04, 2.09900000e+04, 1.95860000e+04, 2.05180000e+04,
       1.96000000e+04, 1.81320000e+04, 2.19390000e+04, 2.08680000e+04,
       2.01980000e+04, 2.08680000e+04, 2.00260000e+04, 2.10710000e+04,
       2.08280000e+04, 1.77970000e+04, 2.05990000e+04, 1.83720000e+04,
       2.11100000e+04, 1.87600000e+04, 1.93230000e+04, 2.09920000e+04,
      

In [38]:
df_y_pred = pd.DataFrame(linreg.predict(X_test), columns = ['price'])

In [39]:
df_pred = df_nan.join(df_y_pred)

In [ ]:
df_pred.head(35)

In [ ]:
df_pred.describe()

Something has gone really wrong here, the mean and standard deviation for price are huge, they must contain very large velues - the smart car is super expensive for example... I dont know why this has happened 

In [ ]:
df_pred_cleaned = df_pred[df_pred.carmake != 'smart']

In [ ]:
df_pred_cleaned.describe()

Much more realistic - maybe it is because there are no smart cars in the training data? doesnt explain the super high value completly. 